In [33]:
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier

from operator import itemgetter

The goal of this task is to predict the "StarPlayerBrawler" of the WinningTeam in the mode "BrawlBall". 

1. I will read out only the winning team (The Starplayer can only come from the WInning Team). 
2. The Attributes "WinningTeam", "map", the 3 "Winners" and the "StarPlayerBrawler" of the match should be visualised as a table.
3. I will create a model to predict the "StarPlayerBrawler" out of the 3 players of the WInningTeam
4. Evaluation

# Reading the Dataset

In [2]:
df = pd.read_csv("rawdata.csv")
df.head(5)

,WinningTeam,BattleTime,playerTag,mode,map,result,StarPlayerTag,StarPlayerBrawler,Brawler1Name,Brawler1Tag,Brawler2Name,Brawler2Tag,Brawler3Name,Brawler3Tag,Brawler4Name,Brawler4Tag,Brawler5Name,Brawler5Tag,Brawler6Name,Brawler6Tag
0,1,20200530T122433.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,BEA,#222VJU0CU,BROCK,#20C2RRRGY,EMZ,#Y220JJJ9
1,1,20200530T121808.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,GENE,#2R9YVLQY,MAX,#22L8CLYQU,PENNY,#28U009JPY
2,1,20200530T121207.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,GENE,#8YVQRQP,BROCK,#9QJP9LP,SPIKE,#98VJCL2PQ
3,1,20200530T120359.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PIPER,#VGR0CJ29,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,JACKY,#2UPURLUG2,FRANK,#2UCV2R0LV,POCO,#22RURYRV0
4,1,20200530T120028.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PIPER,#VGR0CJ29,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,PIPER,#PUG8CR80,BROCK,#98JQVGLP9,JACKY,#8822ULJ9Q


# Appending Winners

Since Brawlball is 3 vs 3 mode, we only need the Winners. For each row, i will append the BrawlerNames only of the Winner group:

In [41]:
WinnerTeam = []

for index, row in df.iterrows():
    winners = []
    
    if row["WinningTeam"] == 1 and row["result"] != 0:
        winners.append(row["Brawler1Name"])
        winners.append(row["Brawler2Name"])
        winners.append(row["Brawler3Name"])
        
    
    elif row["result"] != 0:
        winners.append(row["Brawler4Name"])
        winners.append(row["Brawler5Name"])
        winners.append(row["Brawler6Name"])
    
    if len(winners) != 0:
        winners = sorted(winners)
        WinnerTeam.append(winners)


print(WinnerTeam)

#combinedWinnerTeam = []

for i in range(len(WinnerTeam)):
    P1 = WinnerTeam[i][0]
    P2 = WinnerTeam[i][1]
    P3 = WinnerTeam[i][2]

print(P1,P2,P3)
    
 

[['BEA', 'GENE', 'PIPER'], ['BEA', 'GENE', 'PIPER'], ['BEA', 'GENE', 'PIPER'], ['BEA', 'GENE', 'PIPER'], ['BEA', 'GENE', 'PIPER'], ['BEA', 'GENE', 'PIPER'], ['BEA', 'GENE', 'PIPER'], ['BEA', 'GENE', 'PIPER'], ['BEA', 'GENE', 'PIPER'], ['GENE', 'SANDY', 'SPIKE'], ['MAX', 'PAM', 'PIPER'], ['BEA', 'BROCK', 'MR. P'], ['BEA', 'MR. P', 'SPIKE'], ['BEA', 'BROCK', 'MR. P'], ['BEA', 'BROCK', 'MR. P'], ['BEA', 'BROCK', 'MR. P'], ['BEA', 'BROCK', 'MR. P'], ['BEA', 'BROCK', 'MR. P'], ['BEA', 'BROCK', 'MR. P'], ['BEA', 'BROCK', 'MR. P'], ['BEA', 'BROCK', 'MR. P'], ['BEA', 'BROCK', 'MR. P'], ['BEA', 'BROCK', 'MR. P'], ['BROCK', 'MR. P', 'PAM'], ['BEA', 'MR. P', 'PAM'], ['BEA', 'JACKY', 'MORTIS'], ['BEA', 'BROCK', 'MR. P'], ['BEA', 'BROCK', 'MR. P'], ['BEA', 'JACKY', 'PAM'], ['EMZ', 'JESSIE', 'NITA'], ['EMZ', 'JACKY', 'SANDY'], ['EMZ', 'JESSIE', 'MORTIS'], ['BEA', 'BROCK', 'PAM'], ['BEA', 'FRANK', 'NITA'], ['BEA', 'BIBI', 'NITA'], ['BIBI', 'BROCK', 'NITA'], ['BIBI', 'JACKY', 'SANDY'], ['BIBI', 'JACKY

# Creating Table (WinningTeam, map, 3 Winners, Starplayer)

In [42]:
dfFinal = pd.DataFrame(columns=["WinningTeam", "map", "P1", "P2", "P3"])
dfFinal["WinningTeam"] = df["WinningTeam"]
dfFinal["map"] =df["map"]
dfFinal["P1"] = P1
dfFinal["P2"] = P2
dfFinal["P3"] = P3
dfFinal["StarPlayerBrawler"] = df["StarPlayerBrawler"]


# encode maps (copy pasted from Marc)
encoding = LabelEncoder()
encoding.fit(dfFinal["map"].values)
dfFinal["map"] = encoding.transform(dfFinal["map"].values)

dfFinal.head(10)

,WinningTeam,map,P1,P2,P3,StarPlayerBrawler
0,1,0,BO,CARL,PIPER,BEA
1,1,0,BO,CARL,PIPER,BEA
2,1,0,BO,CARL,PIPER,BEA
3,1,0,BO,CARL,PIPER,PIPER
4,1,0,BO,CARL,PIPER,PIPER
5,1,0,BO,CARL,PIPER,GENE
6,2,0,BO,CARL,PIPER,PIPER
7,2,0,BO,CARL,PIPER,PIPER
8,2,0,BO,CARL,PIPER,GENE
9,1,11,BO,CARL,PIPER,SANDY


# Creating Model to somehow predict, who's going to be StarPlayer out of the 3 winning players (maybe with stats like accurays, recall and precision?) 

# Evaluation (and maybe Visualization) of Model